## Import

In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.utils import shuffle
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

import math
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
import missingno as msno

from collections import Counter

In [41]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (39.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


## csv to parquet

In [2]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [3]:
csv_to_parquet('/content/drive/MyDrive/dacon/data/train.csv', 'train')
#csv_to_parquet('/content/drive/MyDrive/dacon/data/test.csv', 'test')

train Done.


## 데이터 불러오기

id = 아이디

base_date = 날짜

day_of_week = 요일

base_hour = 시간대

road_in_use = 도로사용여부

lane_count = 차로수

road_rating = 도로등급

multi_linked = 중용구간 여부

connect_code = 연결로 코드

maximum_speed_limit = 최고속도제한

weight_restricted = 통과제한하중

height_restricted = 통과제한높이

road_type = 도로유형

start_latitude = 시작지점의 위도

start_longtitude = 시작지점의 경도

start_tum_restricted = 시작지점의 회전제한

end_latitude = 도착지점의 위도

end_longtitude = 도착지점의 경도

end_tum_restricted = 도착지점의 회전제한

road_name = 도로명

start_node_name = 시작지점명

end_node_name = 도착지점명

vehicle_restricted = 통과제한차량

target = 평균속도(km)

In [4]:
train = pd.read_parquet('./train.parquet')
#test = pd.read_parquet('./test.parquet')

In [5]:
train.shape

(4701217, 23)

In [6]:
train_df = train.copy()

In [7]:
train['month']=train['base_date']
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0,20220623
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0,20220728
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0,20211010
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0,20220311
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0,20211005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0,20211104
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0,20220331
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0,20220613
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0,20211020


## column 추가(월)

In [8]:
#train데이터에 추가
train['month'] = train['month'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [9]:
train['month'] = train['month'].dt.month

In [10]:
train=train.sort_values('base_date')

In [11]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,진동1로,33.518987,126.568861,있음,동중입구,33.519726,126.571488,없음,37.0,9
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,모살물교,33.312509,126.703863,없음,한남교,33.312449,126.696173,없음,36.0,9
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,43.0,9
4543127,TRAIN_4543127,20210901,수,16,1,107,-,0,0,60.0,...,서중2교,33.331742,126.676431,없음,한남교차로,33.325096,126.665698,없음,55.0,9
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,월산2교,33.260056,126.490987,없음,월산2교,33.259995,126.491306,없음,53.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,신양약방,33.251716,126.425612,없음,중문한의원,33.251884,126.423200,없음,21.0,7
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,제2거로교,33.508464,126.558231,없음,거로사거리,33.507349,126.556173,있음,36.0,7
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,신성교회,33.253074,126.506393,없음,서호2차현대맨션203동,33.252183,126.506069,없음,24.0,7
3749761,TRAIN_3749761,20220731,일,1,2,103,일반국도16호선,0,0,60.0,...,서홍교,33.264112,126.554043,없음,서흥동사무소,33.263685,126.550978,없음,37.0,7


In [ ]:
# month_dummies = pd.get_dummies(train['month'])
# month_dummies

,1,2,3,5,6,7,9,10,11,12
430379,0,0,0,0,0,0,1,0,0,0
369019,0,0,0,0,0,0,1,0,0,0
1132647,0,0,0,0,0,0,1,0,0,0
4543127,0,0,0,0,0,0,1,0,0,0
3428956,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2053638,0,0,0,0,0,1,0,0,0,0
2748124,0,0,0,0,0,1,0,0,0,0
2447938,0,0,0,0,0,1,0,0,0,0
3749761,0,0,0,0,0,1,0,0,0,0


In [ ]:
# train=pd.concat([train, month_dummies], axis=1)
# train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,1,2,3,5,6,7,9,10,11,12
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,0,0,0,0,0,0,1,0,0,0
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,0,0,0,0,0,0,1,0,0,0
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,0,0,0,0,0,0,1,0,0,0
4543127,TRAIN_4543127,20210901,수,16,1,107,-,0,0,60.0,...,0,0,0,0,0,0,1,0,0,0
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,0,0,0,0,0,1,0,0,0,0
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,0,0,0,0,0,1,0,0,0,0
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,0,0,0,0,0,1,0,0,0,0
3749761,TRAIN_3749761,20220731,일,1,2,103,일반국도16호선,0,0,60.0,...,0,0,0,0,0,1,0,0,0,0


## column추가(평일 0, 주말 1 나누기)

In [12]:
workday = ['월', '화', '수', '목', '금']
playday = ['토', '일']

weekend=[]

for row in train['day_of_week']:
    if row in workday:
        weekend.append(0)   # 평일 0
    elif row in playday:
        weekend.append(1)   # 주말 1

train['weekend'] = weekend

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4701217 entries, 430379 to 4680098
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            object 
 3   base_hour              int64  
 4   lane_count             int64  
 5   road_rating            int64  
 6   road_name              object 
 7   multi_linked           int64  
 8   connect_code           int64  
 9   maximum_speed_limit    float64
 10  vehicle_restricted     float64
 11  weight_restricted      float64
 12  height_restricted      float64
 13  road_type              int64  
 14  start_node_name        object 
 15  start_latitude         float64
 16  start_longitude        float64
 17  start_turn_restricted  object 
 18  end_node_name          object 
 19  end_latitude           float64
 20  end_longitude          float64
 21  end_turn_restricted    object 
 22  target       

In [14]:
train_df=train_df.sort_values('base_date')

In [17]:
train['base_hour'].unique()

array([16, 11,  4,  9,  0, 13, 15, 10, 14, 23, 17,  3,  1,  2,  6, 19, 21,
        7,  8, 12, 22, 20,  5, 18])

In [ ]:
#start_node, end_node 상관계수 알아보기(eda)


In [ ]:
# column_names = train.columns.values.tolist()

# for i in column_names:
#     print(f"{i} = {train[i].unique()}")

In [19]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month,weekend
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,33.518987,126.568861,있음,동중입구,33.519726,126.571488,없음,37.0,9,0
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,33.312509,126.703863,없음,한남교,33.312449,126.696173,없음,36.0,9,0
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,43.0,9,0
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,33.260056,126.490987,없음,월산2교,33.259995,126.491306,없음,53.0,9,0
2608463,TRAIN_2608463,20210901,수,11,2,103,일반국도12호선,0,0,60.0,...,33.493546,126.445105,없음,도평마을,33.493184,126.442008,없음,19.0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457292,TRAIN_3457292,20220731,일,4,1,107,관광단지로,0,0,30.0,...,33.251003,126.412163,없음,선방석재개발(주),33.251812,126.412607,없음,39.0,7,1
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,33.251716,126.425612,없음,중문한의원,33.251884,126.423200,없음,21.0,7,1
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,33.508464,126.558231,없음,거로사거리,33.507349,126.556173,있음,36.0,7,1
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,33.253074,126.506393,없음,서호2차현대맨션203동,33.252183,126.506069,없음,24.0,7,1


In [20]:
train[train.duplicated()]

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month,weekend


### 시간대

In [20]:
dawn = [0,1,2,3,4,5]
morning = [6,7,8,9,10,11]
daytime = [12,13,14,15,16,17]
night = [18,19,20,21,22,23]

hour_label = []

for row in train['base_hour']:
    if row in dawn:
        hour_label.append(0)   # 새벽 0
    elif row in morning:
        hour_label.append(1)   # 오전 1
    elif row in daytime:
        hour_label.append(2)   # 오후 2
    elif row in night:
        hour_label.append(3)   # 저녁 3

train['hour_label'] = hour_label
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month,weekend,hour_label
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,126.568861,있음,동중입구,33.519726,126.571488,없음,37.0,9,0,2
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,126.703863,없음,한남교,33.312449,126.696173,없음,36.0,9,0,1
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,126.326551,없음,애월입구,33.462677,126.330152,없음,43.0,9,0,0
4543127,TRAIN_4543127,20210901,수,16,1,107,-,0,0,60.0,...,126.676431,없음,한남교차로,33.325096,126.665698,없음,55.0,9,0,2
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,126.490987,없음,월산2교,33.259995,126.491306,없음,53.0,9,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,126.425612,없음,중문한의원,33.251884,126.423200,없음,21.0,7,1,3
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,126.558231,없음,거로사거리,33.507349,126.556173,있음,36.0,7,1,2
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,126.506393,없음,서호2차현대맨션203동,33.252183,126.506069,없음,24.0,7,1,2
3749761,TRAIN_3749761,20220731,일,1,2,103,일반국도16호선,0,0,60.0,...,126.554043,없음,서흥동사무소,33.263685,126.550978,없음,37.0,7,1,0


## 공휴일

In [23]:
holi = [20210920, 20210921, 20210922, 20211004, 20211011, 20211009, 20211003, 20211225,
        20220101, 20220131, 20220201, 20220202, 20220301, 20220309, 20220501, 20220505,
        20220508, 20220601, 20220606, 20220717]

holiday = []

for row in train['base_date']:
    if row in holi:
        holiday.append(1)   # 공휴일이면 1
    else:
        holiday.append(0)   # 공휴일 아니면 0

train['holiday'] = holiday
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month,weekend,hour_label,holiday
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,있음,동중입구,33.519726,126.571488,없음,37.0,9,0,2,0
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,없음,한남교,33.312449,126.696173,없음,36.0,9,0,1,0
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,없음,애월입구,33.462677,126.330152,없음,43.0,9,0,0,0
4543127,TRAIN_4543127,20210901,수,16,1,107,-,0,0,60.0,...,없음,한남교차로,33.325096,126.665698,없음,55.0,9,0,2,0
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,없음,월산2교,33.259995,126.491306,없음,53.0,9,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,없음,중문한의원,33.251884,126.423200,없음,21.0,7,1,3,0
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,없음,거로사거리,33.507349,126.556173,있음,36.0,7,1,2,0
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,없음,서호2차현대맨션203동,33.252183,126.506069,없음,24.0,7,1,2,0
3749761,TRAIN_3749761,20220731,일,1,2,103,일반국도16호선,0,0,60.0,...,없음,서흥동사무소,33.263685,126.550978,없음,37.0,7,1,0,0


## 날씨

### 장마

In [26]:
rainday = [20220621,20220622,20220623,20220624,20220625,20220626,20220627,20220628,20220629,20220630,
           20220701,20220702,20220703,20220704,20220705,20220706,20220707,20220708,20220709,20220710,
           20220711,20220712,20220713,20220714,20220715,20220716,20220717,20220718,20220719,20220720,
           20220721,20220722,20220723,20220724]

rain_season = []

for i in train['base_date']:
    if i in rainday:
        rain_season.append(1)
    else:
        rain_season.append(0)

train['rain_season'] = rain_season
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,month,weekend,hour_label,holiday,rain_season
430379,TRAIN_0430379,20210901,수,16,3,106,지방도1132호선,0,0,70.0,...,동중입구,33.519726,126.571488,없음,37.0,9,0,2,0,0
369019,TRAIN_0369019,20210901,수,11,1,103,일반국도16호선,0,0,50.0,...,한남교,33.312449,126.696173,없음,36.0,9,0,1,0,0
1132647,TRAIN_1132647,20210901,수,4,2,103,일반국도12호선,0,0,80.0,...,애월입구,33.462677,126.330152,없음,43.0,9,0,0,0,0
4543127,TRAIN_4543127,20210901,수,16,1,107,-,0,0,60.0,...,한남교차로,33.325096,126.665698,없음,55.0,9,0,2,0,0
3428956,TRAIN_3428956,20210901,수,9,1,103,일반국도16호선,0,0,50.0,...,월산2교,33.259995,126.491306,없음,53.0,9,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,일,21,1,107,중문로,0,0,50.0,...,중문한의원,33.251884,126.423200,없음,21.0,7,1,3,0,0
2748124,TRAIN_2748124,20220731,일,12,3,107,연삼로,0,0,70.0,...,거로사거리,33.507349,126.556173,있음,36.0,7,1,2,0,0
2447938,TRAIN_2447938,20220731,일,12,2,107,경찰로,0,0,60.0,...,서호2차현대맨션203동,33.252183,126.506069,없음,24.0,7,1,2,0,0
3749761,TRAIN_3749761,20220731,일,1,2,103,일반국도16호선,0,0,60.0,...,서흥동사무소,33.263685,126.550978,없음,37.0,7,1,0,0,0


In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4701217 entries, 430379 to 4680098
Data columns (total 28 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            object 
 3   base_hour              int64  
 4   lane_count             int64  
 5   road_rating            int64  
 6   road_name              object 
 7   multi_linked           int64  
 8   connect_code           int64  
 9   maximum_speed_limit    float64
 10  vehicle_restricted     float64
 11  weight_restricted      float64
 12  height_restricted      float64
 13  road_type              int64  
 14  start_node_name        object 
 15  start_latitude         float64
 16  start_longitude        float64
 17  start_turn_restricted  object 
 18  end_node_name          object 
 19  end_latitude           float64
 20  end_longitude          float64
 21  end_turn_restricted    object 
 22  target       

In [ ]:
A=train[train['road_name']=='-'].index
train.drop(A,axis='index',inplace=True)

## EDA

### 요일 분포

In [ ]:
train.isnull().sum()

id                       0
base_date                0
day_of_week              0
base_hour                0
lane_count               0
road_rating              0
road_name                0
multi_linked             0
connect_code             0
maximum_speed_limit      0
vehicle_restricted       0
weight_restricted        0
height_restricted        0
road_type                0
start_node_name          0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_node_name            0
end_latitude             0
end_longitude            0
end_turn_restricted      0
target                   0
month                    0
weekend                  0
dtype: int64

In [ ]:
train = train.drop(['id', 'base_date', 'day_of_week', 'road_name', 'multi_linked', 'connect_code', 'vehicle_restricted', 'height_restricted', 'start_node_name', 'end_node_name', 'target'], axis=1)

## LightGBM

In [ ]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

    print(i, le.classes_)

start_turn_restricted ['없음' '있음']
end_turn_restricted ['없음' '있음']


In [ ]:
int_col = ['month']
for i in int_col:
    le_int = LabelEncoder()
    le_int=le_int.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le_int.classes_: 
            le_int.classes_ = np.append(le_int.classes_, label)
    test[i]=le_int.transform(test[i])

    print(i, le_int.classes_)

In [ ]:
# 요일, 시간대, 차로수, 도로등급, 중용구간 여부, 연결로 코드, 최고속도제한, 통과제한하중, 통과제한높이
# 도로유형, 시작지점위도, 시작지점경도, 시작지점의 회전제한 유무
# 도착지점위도, 도착지점경도, 도착지점의 회전제한 유무
# 16개
train.head(7)

,base_hour,lane_count,road_rating,maximum_speed_limit,weight_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,month,weekend
430379,16,3,106,70.0,0.0,3,33.518987,126.568861,1,33.519726,126.571488,0,9,0
369019,11,1,103,50.0,0.0,0,33.312509,126.703863,0,33.312449,126.696173,0,9,0
1132647,4,2,103,80.0,0.0,0,33.462214,126.326551,0,33.462677,126.330152,0,9,0
4543127,16,1,107,60.0,0.0,0,33.331742,126.676431,0,33.325096,126.665698,0,9,0
3428956,9,1,103,50.0,32400.0,3,33.260056,126.490987,0,33.259995,126.491306,0,9,0
2608463,11,2,103,60.0,0.0,0,33.493546,126.445105,0,33.493184,126.442008,0,9,0
3058655,0,2,103,80.0,0.0,0,33.280963,126.356788,0,33.280115,126.361832,0,9,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4701217 entries, 430379 to 4680098
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_hour              int64  
 1   lane_count             int64  
 2   road_rating            int64  
 3   maximum_speed_limit    float64
 4   weight_restricted      float64
 5   road_type              int64  
 6   start_latitude         float64
 7   start_longitude        float64
 8   start_turn_restricted  int64  
 9   end_latitude           float64
 10  end_longitude          float64
 11  end_turn_restricted    int64  
 12  month                  int64  
 13  weekend                int64  
dtypes: float64(6), int64(8)
memory usage: 538.0 MB


In [ ]:
column_names = train.columns.values.tolist()

for i in column_names:
    print(f"{i} = {train[i].nunique()}")

base_hour = 24
lane_count = 3
road_rating = 3
maximum_speed_limit = 6
weight_restricted = 4
road_type = 2
start_latitude = 586
start_longitude = 586
start_turn_restricted = 2
end_latitude = 586
end_longitude = 586
end_turn_restricted = 2
month = 10
weekend = 2


### 데이터 분포 확인 eda

In [ ]:
# fig, ax = plt.subplots(7, 2, figsize=(12, 50)) 

# # id 변수(count==0인 경우)는 제외하고 분포를 확인합니다.
# count = 1
# columns = train.columns
# for row in range(14):
#     for col in range(2):
#         sns.kdeplot(data=train[columns[count]], ax=ax[row][col])
#         ax[row][col].set_title(columns[count], fontsize=15)
#         count += 1
#         if count == 15 :
#             break

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
data = train
target = train_df['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3760973, 14)
(3760973,)
(940244, 14)
(940244,)


### 훈련

In [ ]:
model = LGBMRegressor(random_state=42)

learning_rate : 학습률(작을수록 학습속도가 느려짐), default 0.1

max_depth:트리의 깊이 default 1

num_leaves: 하나의 트리가 가질 수 있는 최대 리프 개수 default 31

n_estimators: default 100


default model

In [ ]:
# X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
# model_default = model = LGBMRegressor(random_state=42)

In [ ]:
# model_default.fit(X_train, Y_train)

In [ ]:
# pred_default= model_default.predict(X_test)
# mae = mean_absolute_error(pred_default, Y_test)
# print(mae)

In [ ]:
param_lgb = {"learning_rate" : [0.5],
             "max_depth" : [15,16,17,18],
             "num_leaves" : [100, 150, 200],
             "n_estimators" : [50, 100, 150],
              }

In [ ]:
from sklearn.model_selection import GridSearchCV

**neg_mean_absolute_error**

하지만 cross_val_score같은 경우, 인자에 'neg'라는 단어를 붙여주어야 하는데 이는 Negative(음수)를 뜻한다. 왜 음수를 붙여주는 걸까? 그 이유는 바로 cross_val_score나 GridSearchCV 함수의 경우 score를 출력해줄 때 가장 높은 수치의 score를 출력하도록 되어있기 때문이다. 예를 들어, cross_val_score로 교차검증을 3번 실시한 후 각각의 MSE값이 [3, 5, 7] 이라는 결과값이 나왔다. 이 때 cross_val_score의 가장 높은 수치의 score를 출력하는 특성대로 한다면 MSE값은 낮을수록 예측성능이 좋다는 의미임에도 불구하고 '7'이라는 가장 안좋은 MSE값을 출력하게 된다. 따라서 결과값들의 순위를 바꿔주기 위해서 neg(negative=음수)를 붙여 -1을 곱해주게 되는 것이다.

https://techblog-history-younghunjo1.tistory.com/105

In [ ]:
gscv_lgb = GridSearchCV (estimator = model, param_grid = param_lgb, scoring='neg_mean_absolute_error', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_lgb.fit(x_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=100; total time=  10.8s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=100; total time=  11.1s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=100; total time=  12.7s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=150; total time=  12.0s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=150; total time=  12.1s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=150; total time=  16.2s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=200; total time=  17.0s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=200; total time=  13.4s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=50, num_leaves=200; total time=  12.4s
[CV] END learning_rate=0.5, max_depth=15, n_estimators=100, num_leaves=100; total time=  18.

GridSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_jobs=1,
             param_grid={'learning_rate': [0.5], 'max_depth': [15, 16, 17, 18],
                         'n_estimators': [50, 100, 150],
                         'num_leaves': [100, 150, 200]},
             scoring='neg_mean_absolute_error', verbose=2)

history = LSTM_SGD.fit(x_train_set, Y_train,
                         epochs = num_epochs,
                         batch_size=512,
                         validation_split=0.2,
                         verbose=1,
                         callbacks=callbacks1)

params : 적용된 개별 하이퍼 파라미터의 값

mean_test_score : 세트에 대해 수행된 성능들의 평균값

rank_test_score : mean_test_score을 바탕으로 평가 순위

split_test_score : 각 세트별 정확도

In [ ]:
scores_df = pd.DataFrame(gscv_lgb.cv_results_)
scores_df[['params', 'mean_test_score','rank_test_score', 'split0_test_score', 'split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.233835,2,-13.237291,-13.232042,-13.232171
1,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.238375,6,-13.243006,-13.235888,-13.236231
2,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.242500,12,-13.246905,-13.240856,-13.239738
3,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.243051,15,-13.247075,-13.240675,-13.241401
4,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.251179,21,-13.255877,-13.248986,-13.248674
5,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.258703,27,-13.264605,-13.257027,-13.254477
6,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.251780,24,-13.256595,-13.249349,-13.249398
7,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.262018,30,-13.266125,-13.260098,-13.259831
8,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",-13.272874,34,-13.278493,-13.271266,-13.268864
9,"{'learning_rate': 0.5, 'max_depth': 16, 'n_est...",-13.233950,3,-13.237634,-13.232003,-13.232213


In [ ]:
print('GridSearchCV 최적 파라미터:', gscv_lgb.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(gscv_lgb.best_score_))

GridSearchCV 최적 파라미터: {'learning_rate': 0.5, 'max_depth': 17, 'n_estimators': 50, 'num_leaves': 100}
GridSearchCV 최고 정확도:-13.2335


In [ ]:
gscv_lgb.cv_results_

{'mean_fit_time': array([ 9.41384816, 11.65059813, 12.31652633, 15.57952126, 16.50091664,
        18.6830558 , 22.75455586, 24.233416  , 23.60346349, 10.06589675,
         9.63647159, 11.1777606 , 15.89824033, 16.79343144, 16.63204082,
        22.13485344, 23.49961154, 24.14846055,  9.0759844 , 10.90474248,
         9.91260974, 16.38590717, 16.14050007, 18.1928219 , 21.24304183,
        23.36257219, 24.78688534,  9.01756779,  9.91703192, 11.19298577,
        14.82999094, 15.87240505, 16.63118641, 21.62055008, 23.07766207,
        22.83802001]),
 'std_fit_time': array([0.22235698, 1.96544463, 1.96241605, 0.30016024, 0.1594537 ,
        2.5307419 , 1.63753958, 1.88290598, 0.46754525, 1.47781152,
        0.36584838, 1.83573241, 1.32341343, 0.49310971, 0.08427437,
        1.633461  , 1.92187051, 1.7056379 , 0.06058887, 1.80575817,
        0.18827113, 2.09931124, 0.21153942, 1.94567885, 0.939264  ,
        1.54680209, 1.89033765, 0.3804572 , 0.33165025, 1.77033023,
        0.34514286, 0.420

In [ ]:
# params = {'learning_rate' : 0.6,
#           'max_depth' : 50,
#           'n_estimator' : 300,
#           'num_leaves' : 50}

In [ ]:
LR = lgb.LGBMRegressor(max_depth = 17,
                       num_leaves = 100,
                       learning_rate = 0.6,
                       n_estimators = 300).fit(x_train, y_train)

pred = LR.predict(x_test)
mae = mean_absolute_error(pred, y_test)
print(mae)

13.275746345880478


In [ ]:
# pred

In [ ]:
{'learning_rate': 0.5, 'max_depth': 17, 'n_estimators': 50, 'num_leaves': 100}

In [ ]:
# pred = model.predict(x_test)

In [ ]:
# pred

### MSE